<p><font size="4">  MOOC: Understanding queues</font></p>
<p><font size="4">  Python lab
</p></br>
<p><font size="4">  Week V: Multi server systems </p></br>

----------------------------

This lab focusses on multiserver queues. Numerical computations are peformed that illustrate the results obtained in the problems that we just studied.

---------

**1)** Let us consider an M/M/C/C queue with offered load $\rho$. Complete the code of function **p(rho,C)** that returns the vector of length $C+1$ that represents the stationary distribution of an M/M/C/C queue with $C$ servers 

In [1]:
%matplotlib inline              
from pylab import *          

In [27]:
# Calculation of the stationary distribution of an M/M/C/C
def p(rho,C):
    pi_   = ones(C+1)
    for k in range(C):
        #####################
        # supply the expression of pi_[k+1]
        # in terms of pi_[k]
        pi_[k+1] = (1/(k+1)) * pi_[k]
        #####################
    pi_ = pi_/sum(pi_)
    return pi_

#----------
V1 = p(1,4)[0]
V1

0.36923076923076925

**2)** Complete the code of the function **Eb(rho,C)** that implements 
the computation of the loss probability (Erlang B formula). Check that for the problem of the videoconference server where $\lambda=1$, $\mu=1$ and $C=4$ you find the same result as in the problem. 

In [30]:
from math import factorial
#############
# Complete the code of Erlang B formula 
def Eb(rho,C):
    return round( (( rho**C) / factorial(C) ) / sum([(rho**i)/factorial(i) for i in range(C)]), 5)
#############
lambda_, mu, C = 1., 1., 4
V2 = Eb(lambda_/mu,C)

**3)** Using the curves below that represent the loss probability $E_b$ vs the offered load, give the minimum values of $C$ for which the loss probability is lower than 3% and .5% respectively for $\rho=1$Erlang. Note that you can change the value of $C$ in the figure below by moving around the slider on top of it.

In [31]:
# Importing ipywidgets for interactive plotting
from ipywidgets import *

In [33]:
rcParams['figure.figsize'] = 10,7
def plt_Eb(C):
    # plot of rho -->Eb(rho,c)
    C_max   = 20
    rho_max = 4
    ymin    = 1.0e-7
    ymax    = 60
    rho     = linspace(0,rho_max,100)

    plot(rho, [100*Eb(r,C) for r in rho])
    grid('on')
    xlabel(r"Offered Load $\rho$", fontsize=20)
    ylabel(r"$E_B (\%)$", fontsize=20)
    title("Loss probability", fontsize=20)
    def point_loss(x,y,color,label):
        # point rho=x and loss_probability=y%
        semilogy(x,y,'o',color=color,label=label)
        semilogy((0,rho_max),(y,y),'r',linewidth=.4)
    semilogy((1,1),(ymin,ymax),'r',linewidth=.4)     
    # point of loss probability 1%
    point_loss(1,3,color='b',label=r'$E_B=3\%$')
    point_loss(1,.5,color='g',label=r'$E_B=0.5\%$')
    legend(fontsize=20)
    axis(xmin=0,xmax=rho_max,ymin=ymin,ymax=ymax)
    show()

# interactive plot of rho -->Eb(rho,c) for varying c
interact(plt_Eb, C=(1,10,1))

#######################
# supply the minimum values of C such that Eb<3% and Eb<.5%
V3 = 4   # min value of C such that Eb(1,C)<3%
V4 = 5   # min value of C such that Eb(1,C)<.5%
#######################

interactive(children=(IntSlider(value=5, description='C', max=10, min=1), Output()), _dom_classes=('widget-int…

**4)** Now, we are going to extend the results of the video conferencing system problem to the case where there can be subscribers and occasional clients and $C\geq 1$.

For instance, in the case $C=3$ states of the system are in the form $(a,b)$ with $0\leq a,b\leq 3$.

<img src="video_conf_C=3.png" height=500, width=700>

In this case, putting balance equations in the form $\pi {\bf Q}=0$ yields a generator matrix ${\bf Q}$ in the form

$$
{\bf Q} =\\
\begin{pmatrix}
-(\lambda+\lambda') &\lambda'&0&0&\lambda&0&0&0&0&0\\
\mu&-(\lambda+\lambda'+\mu)&\lambda'&0&0&\lambda&0&0&0&0\\
0&2\mu&-(\lambda+\lambda'+2\mu)&\lambda'&0&0&\lambda&0&0&0\\
0&0&3\mu&-(\lambda+3\mu)&0&0&\lambda&0&0&0\\
\mu&0&0&0&-(\lambda+\lambda'+\mu)&\lambda'&0&\lambda&0&0\\
0&\mu&0&0&\mu&-(\lambda+\lambda'+2\mu)&\lambda'&0&\lambda&0\\
0&0&\mu&0&0&2\mu&-(\lambda+3\mu)&0&\lambda&0\\
0&0&0&0&2\mu&0&0&-(\lambda+\lambda'+2\mu)&\lambda'&\lambda\\
0&0&0&0&0&2\mu&0&\mu&-(\lambda+3\mu)&\lambda\\
0&0&0&0&0&0&0&3\mu&0&-3\mu\\
\end{pmatrix}
$$

where $[{\bf Q}]_{ij}$ is the entry $(i,j)$ of ${\bf Q}$ that denotes transition intensity from state $(a,b)$ to state $(c,d)$ with $i=(a(C+1)-a(a-1)/2+b$ and $j=c(C+1)-c(c-1)/2+d$ for $i\neq j$ and $[{\bf Q}]_{ii}=-\sum_{j\neq i}[{\bf Q}]_{ij}$. The general implementation of matrix ${\bf Q}$ is given in function **matQ(lambda_s,mu,lambda_o,C)** below where **lambda_s** and **lambda_o** stand for the intensity of arrivals  of subscribers ($\lambda$) and occasional customers ($\lambda'$). Complete the code of the function and check that for $C=3$ we find the expression given above here for ${\bf Q}$. 

In [34]:
def ab_to_i(a,b):
    # index conversion (a,b)-->i
    # (a,b) stands for the state (N,N') 
    # and i for the corresponding line or column index of Q
    # note that matrix and state pairs indexing begin at 0
    return int(a*(C+1)-a*(a-1)/2 + b)

def matQ(lambda_s,mu,lambda_o,C):
    # building Q
    s = int((C+1)**2-C*(C+1)/2)
    Q = zeros(shape=(s,s))
    for a in range(C+1):       # loop over N
        for b in range(C+1-a): # loop over N'
            i = ab_to_i(a,b)   # corresponding index in the matrix Q
            if b>0: # transition (a,b)-->(a,b-1)
                Q[i,ab_to_i(a,b-1)]     = b*mu 
            if a>0: # transition (a,b)-->(a-1,b)
                Q[i,ab_to_i(a-1,b)]     = a*mu 
            if b<C-a: # transition (a,b)-->(a,b+1)
                Q[i,ab_to_i(a,b+1)]     = lambda_o 
                if i<s-1: # transition (a,b)-->(a+1,b)
                    Q[i,ab_to_i(a+1,b)] = lambda_s 
            elif i<s-1: # transition (a,C-a)-->(a+1,C-a-1)
                Q[i,ab_to_i(a+1,b-1)]   = lambda_s      
            ############################
            # Supply the value of Q[i,i]
            Q[i,i] = ...  # setting constraint sum(Q[i,:])=0
            ############################
    return Q

lambda_s,mu,lambda_o,C = 1,1,2,3
matQ(lambda_s,mu,lambda_o,C)

TypeError: float() argument must be a string or a number, not 'ellipsis'

**5)** For $C=4$, let us compute the stationary distribution $\pi_Q$ of the system from equations $\pi_Q{\bf Q}=0$ and $\sum_i \pi_Q[i]=1$ (two techniques are provided below). From $\pi_Q$, compute the blocking probability for $N$ and for $N'$.

In [ ]:
# to solve overdetermined systems of equations Ax=b
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html#numpy.linalg.lstsq
from scipy.linalg import lstsq

In [ ]:
set_printoptions(precision=4)
lambda_s,mu,lambda_o,C = 1,1,2,4
A     = matQ(lambda_s,mu,lambda_o,C).T
m,n   = A.shape
A     = vstack((A,ones(n))) # adding constraint sum(pi[i])=1
b     = zeros(m+1)
b[-1] = 1
# solves equations pi.Q=0 and normalization condition
pi_Q  = lstsq(A,b)[0] 
##########################
# supply the blocking probability for for suscribers 
# and occasional users
def blocking(pi_Q,C):
    #returns blocking probability for suscribers (P_s)
    # and occasional users (P_o)
    P_s = ...
    P_o = ...
    return P_s, P_o
##########################
V5,V6 = blocking(pi_Q,C)
print("The blocking probability for N  is %.3f" %V5)
print('Erlang B formula yields Eb(',lambda_s/mu,',',C,
                          ') = %.3f' %Eb(lambda_s/mu,C))
print("The blocking probability for N' is %.3f" %V6)

Until now, we have considered the case $C=4$. Le t us show the influence of $C$ on blocking probabilities for subscribers and occasional customers. As expected blocking probabilities for subscribers remains lower than that of  occasional customers and both blocking probabilities decrease to 0 as $C$ increases. 

In [ ]:
Cmax = 10
rcParams["figure.figsize"] = 12,9
for C in range(1,Cmax+1):
    A     = matQ(lambda_s,mu,lambda_o,C).T
    m,n   = A.shape
    A     = vstack((A,ones(n))) # adding constraint sum(pi[i])=1
    b     = zeros(m+1)
    b[-1] = 1
    # solves equations pi.Q=0 and normalization condition
    pi_Q  = lstsq(A,b)[0] 
    P_s,P_o = blocking(pi_Q,C)
    semilogy(C,P_s,'or')
    semilogy(C,P_o,'og')
semilogy(C,P_s,'or',label='subscribers')
semilogy(C,P_o,'og',label='occasional')
grid('on')
legend(fontsize=20)
xlabel('C',fontsize=20)
ylabel('Blocking probability',fontsize=20)

# Conclusion

In this lab, we have confirmed the interest of using several servers in queuing systems. At the end of this week you probably have become quite familiar with tweaking the code to deal with a few standard or more advanced queuing systems. 

Here we are! We hope that you found these labs helpful for understanding queues and possibly set up your own simulation experiments of queueing systems.

# Your answers for the exercise

In [35]:
print("---------------------------\n"
      +"RESULTS SUPPLIED FOR LAB 5:\n"
      +"---------------------------")
results = ("V"+str(k) for k in range(1,7))
for x in results:
    try:
        print(x+" = {0:.3f}".format(eval(x)))
    except:
        print(x+": variable is undefined")

---------------------------
RESULTS SUPPLIED FOR LAB 5:
---------------------------
V1 = 0.369
V2 = 0.016
V3 = 4.000
V4 = 5.000
V5: variable is undefined
V6: variable is undefined


In [2]:
import pandas as pd

In [3]:
xml = '<Ticket>
<TicketStart Opcode="33" TicketStart="1" NoSale="0" RecalledTicket="0" VoidTicket="0" TenderPurchase="0" opt_wic_ticket="0" opt_freq_shop="0" return_type="0" recall_ticket_no="0" wic_issue_day="0" wic_issue_year="0" wic_exp_day="0" wic_exp_year="0" wic_issue_month="0" wic_exp_month="0" wic_amount="0" recall_pos_no="0" SelfScanningTicket="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:03" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49029" />
<Location Opcode="96" Function="49" TenderCorrection="0" FuelMergeRecall="0" Invoice="0" Frgn_cust="0" Express_pos="0" Rdh_sale="0" DepositPrevTicket="0" Rdh_return="0" StoreID="008166" POSTermType="0" CheckoutBankNo="11" EFTLocation="00000" POSTermProfile="0" DocType="0" POSTermNo="1" CashierNo="217" wholesale_trs_type="0" InvoiceNo="" InvoiceDate="1900-01-01" InvoiceTime="00:00:00" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:03" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49030" />
<PluSale Opcode="1" PluNo="00007225002075" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="0" QtyIsWeight="0" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="1" Non-RXItem="0" opt_WIC_CVV="0" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="1000" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="1" Price="299" Amount="299" nt_amount="299" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:03" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49031" />
<PluSale Opcode="1" PluNo="00004410010201" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="0" QtyIsWeight="0" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="1" Non-RXItem="0" opt_WIC_CVV="0" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="1004" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="1" Price="279" Amount="279" nt_amount="279" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:05" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49035" />
<PluSale Opcode="1" PluNo="00071514150349" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="0" QtyIsWeight="0" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="1" Non-RXItem="0" opt_WIC_CVV="0" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="1004" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="1" Price="299" Amount="299" nt_amount="299" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:11" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49039" />
<PluSale Opcode="1" PluNo="00000000004011" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="1" QtyIsWeight="1" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="0" Non-RXItem="0" opt_WIC_CVV="1" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="4000" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="1150" Price="49" Amount="56" nt_amount="56" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:23" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49040" />
<PluSale Opcode="1" PluNo="00000000004727" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="1" QtyIsWeight="1" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="0" Non-RXItem="0" opt_WIC_CVV="1" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="4000" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="1060" Price="129" Amount="137" nt_amount="137" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:28" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49041" />
<PluSale Opcode="1" PluNo="00000000004727" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="1" QtyIsWeight="1" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="0" Non-RXItem="0" opt_WIC_CVV="1" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="4000" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="380" Price="129" Amount="49" nt_amount="49" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:32" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49043" />
<PluSale Opcode="1" PluNo="00004126876433" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="0" QtyIsWeight="0" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="1" Non-RXItem="0" opt_WIC_CVV="0" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="7000" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="1" Price="249" Amount="249" nt_amount="249" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:40" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49045" />
<PluSale Opcode="1" PluNo="00002550000376" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="0" QtyIsWeight="0" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="1" Non-RXItem="0" opt_WIC_CVV="0" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="1000" MultipleSellUnits="1" ReturnType="0" TaxPointer="0" Qty="1" Price="439" Amount="439" nt_amount="439" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:41" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49046" />
<PluSale Opcode="1" PluNo="00003400000245" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="0" QtyIsWeight="0" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="1" Non-RXItem="0" opt_WIC_CVV="0" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="8004" MultipleSellUnits="1" ReturnType="0" TaxPointer="1" Qty="1" Price="99" Amount="99" nt_amount="99" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:46" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49047" />
<PluSale Opcode="1" PluNo="00003400000245" Subtract="0" Cancel="0" Negative="0" opt_was_cancelled="0" opt_supplier_promotion="0" opt_staff_discountable="1" opt_accept_price_override="0" PriceOverride="0" ManualPrice="0" WeightFromScale="0" QtyIsWeight="0" ChainedPrevItem="0" NonMerchandise="0" StoreCoupon="0" VendorCoupon="0" ItemDiscountable="1" ScannedItem="1" Non-RXItem="0" opt_WIC_CVV="0" opt_manual_tare_weight="0" opt_freq_shop_disc="0" FSPayment="1" DepartmentNo="8004" MultipleSellUnits="1" ReturnType="0" TaxPointer="1" Qty="1" Price="99" Amount="99" nt_amount="99" opt_rss_item_sale="0" BottleDeposit="0" BottleRefund="0" opt_Rx="0" opt_qty_case="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:08:47" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49048" />
<Media Opcode="4" MediaNo="1" Change="0" Subtract="0" Cancel="0" opt_was_cancelled="0" Return="0" opt_MCR_used="0" CouponSale="0" CashBack="0" Type="1" Amount="2025" AccountNumber="00000000000000000000" exp_date="0000" auth_no="00000000" count="1" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:09:12" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49050" />
<Media Opcode="4" MediaNo="1" Change="1" Subtract="0" Cancel="0" opt_was_cancelled="0" Return="0" opt_MCR_used="0" CouponSale="0" CashBack="0" Type="1" Amount="-4" AccountNumber="00000000000000000000" exp_date="0000" auth_no="00000000" count="1" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:09:13" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49055" />
<Tax Opcode="6" taxnumber="1" taxable_amt="198" tax_amt="16" exempt="0" TaxIncl="0" TaxExemptNo="00000000000000000000" FdStmpFrgvTxbl="0" FdStmpFrgvTx="0" NoTaxAmunt="0" TaxRefundAmount="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:09:13" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49056" />
<TicketEnd Opcode="5" TicketTotal="1" VoidTicket="0" SaveTicket="0" RecallTicket="0" SendToQbuster="0" TaxValue="16" ItemsNo="10" Amount="2021" Discount="0" NetAmount="0" nt_discount="0" GrandTotal="615511829" FoodStampSales="0" TailDoNotProcess="0" TailBadRecord="0" TailTrainingMode="0" TailPosOffline="0" TailVoidTicket="0" TailReturnTicket="0" TailTV="15" Unique="1266" TailStoreNumber="8166" TailDate="2020-09-20" TailTime="07:09:16" TailCashierNo="217" TailPcNo="1" TailPosNo="1" TailTicketNumber="1536" TailSequenceNumber="49096" />
</Ticket>'

SyntaxError: EOL while scanning string literal (<ipython-input-3-c0b417fd4c11>, line 1)

In [4]:
path = 'C:/Users/1770858/Desktop/test.xml'
pd.read_xml(path)


AttributeError: module 'pandas' has no attribute 'read_xml'

In [ ]:
pd.read_x